In [1]:
import numpy as np
from numpy.linalg import eig
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound
from celmech.disturbing_function import laplace_b as b
import math

In [2]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [3]:
def make_sim(alpha, mass, ecc, pom):
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)
    mu1, mu2, mu3 = mass
    ecc1, ecc2, ecc3 = ecc
    pomega1, pomega2, pomega3 = pom
    
    # start simulation
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    # add star, planet 1, planet 2
    sim.add(m=1.)
    sim.add(m=mu1, P=P1, e=ecc1, pomega=pomega1)
    sim.add(m=mu2, P=P2, e=ecc2, pomega=pomega2)
    sim.add(m=mu3, P=P3, e=ecc3, pomega=pomega3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.ri_whfast.safe_mode = 0
    sim.dt = sim.particles[1].P/12
    sim.collision = "direct"
    
    return sim

In [4]:
def run(sim):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    
    ec13 = 1-ps[1].a/ps[3].a
    m_tot = ps[1].m + ps[2].m + ps[3].m
    Tsec = 2*np.pi/(1/2*m_tot/ps[0].m/ec13**2)*ps[1].P
    
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
    ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
    ec13 = alpha13**(-1/2)*(1-alpha13)
    
    Delta = (ec12 - ec23)/ec13
    mu = (mu3-mu1)/(mu1+mu3)
    a = (1-Delta)**3*(3+Delta)
    b = (1+Delta)**3*(3-Delta)
    
    # coefficients for S1'
    c_e12 = -(mu1+mu3)**2*mu1*a + 16*mu2*Delta*mu1**2
    c_e23 = (mu1+mu3)**2*mu3*b + 16*mu2*Delta*mu3**2
    c1, c2, c3 = -c_e12, (c_e12-c_e23), c_e23
    
    # coefficients for S2'
    c_e13 = (mu3-mu1*mu2)*b + (mu1-mu2*mu3)*a
    c_eminus = -16*mu2*Delta
    d1, d2, d3 = -c_e13+c_eminus*mu1, -c_eminus*(mu1+mu3), c_e13+c_eminus*mu3
    
    Nout = 1000
    times = np.linspace(0,2*Tsec, Nout)
    Gx, Gy = np.zeros((Nout,3)), np.zeros((Nout,3))
    S1ax, S1ay, S2ax, S2ay = np.zeros(Nout), np.zeros(Nout), np.zeros(Nout), np.zeros(Nout)
    for i, time in enumerate(times):
        sim.integrate(time)
        Gx[i] = [np.sqrt(2*p.m*np.sqrt(sim.G*ps[0].m*p.a)*(1-np.sqrt(1-p.e**2)))*np.cos(-p.pomega) for p in ps[1:]]
        Gy[i] = [np.sqrt(2*p.m*np.sqrt(sim.G*ps[0].m*p.a)*(1-np.sqrt(1-p.e**2)))*np.sin(-p.pomega) for p in ps[1:]]
        
        S1ax[i] = c1*ps[1].e*np.cos(-ps[1].pomega) + c2*ps[2].e*np.cos(-ps[2].pomega) + c3*ps[3].e*np.cos(-ps[3].pomega)
        S1ay[i] = c1*ps[1].e*np.sin(-ps[1].pomega) + c2*ps[2].e*np.sin(-ps[2].pomega) + c3*ps[3].e*np.sin(-ps[3].pomega)
        S2ax[i] = d1*ps[1].e*np.cos(-ps[1].pomega) + d2*ps[2].e*np.cos(-ps[2].pomega) + d3*ps[3].e*np.cos(-ps[3].pomega)
        S2ay[i] = d1*ps[1].e*np.sin(-ps[1].pomega) + d2*ps[2].e*np.sin(-ps[2].pomega) + d3*ps[3].e*np.sin(-ps[3].pomega)
    
    Sx = T.T @ Gx.T
    Sy = T.T @ Gy.T
    
    G1, G2, G3 = np.sqrt(Gx**2 + Gy**2).T
    S1, S2, S3 = np.sqrt(Sx**2 + Sy**2)
    S1a = np.sqrt(S1ax**2 + S1ay**2)
    S2a = np.sqrt(S2ax**2 + S2ay**2)
    S3a = S3
    
    return times, G1, G2, G3, S1, S2, S3, S1a, S2a, S3a

# Mode Approximation

In [5]:
def calcMode(sim):
    ps = sim.particles
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    m_tot = m1 + m2 + m3
     
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
    ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
    ec13 = alpha13**(-1/2)*(1-alpha13)
    
    Delta = (ec12 - ec23)/ec13
    a = (1-Delta)**3*(3+Delta)
    b = (1+Delta)**3*(3-Delta)
    
    c_e12 = -(mu1+mu3)**2*mu1*a + 16*mu2*Delta*mu1**2
    c_e23 = (mu1+mu3)**2*mu3*b + 16*mu2*Delta*mu3**2
    c1, c2, c3 = -c_e12, (c_e12-c_e23), c_e23
    
    c_e13 = (mu3-mu1*mu2)*b + (mu1-mu2*mu3)*a
    c_eminus = -16*mu2*Delta
    d1, d2, d3 = -c_e13+c_eminus*mu1, -c_eminus*(mu1+mu3), c_e13+c_eminus*mu3
    
    return 1/np.sqrt(c1**2+c2**2+c3**2) * np.array([c1, c2, c3]), 1/np.sqrt(d1**2+d2**2+d3**2) * np.array([d1, d2, d3])

In [6]:
def calcPsi(sim):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    m_tot = m1 + m2 + m3
     
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
    ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
    ec13 = alpha13**(-1/2)*(1-alpha13)
    
    R2 = np.array([[np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)],
                   [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
                   [np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)]])
    
    Mdiagapprox = R2 @ lsys.Neccentricity_matrix @ R2.T
    psi_theo = 1/2*np.arctan(2*Mdiagapprox[1,0]/(Mdiagapprox[1,1]-Mdiagapprox[0,0]))
    
    Delta = (ec12 - ec23)/ec13
    mu = (mu3-mu1)/(mu1+mu3)
    a = (1-Delta)**3*(3+Delta)
    b = (1+Delta)**3*(3-Delta)
    
    psi_approx = -16*sqrt(mu1*mu2*mu3)*Delta/(a*(mu1+mu3)+16*mu3*Delta-mu2*((mu1+mu3)*b-16*mu3*Delta))
    return psi_theo, psi_approx

In [7]:
from sympy import *
init_printing()
M1, M2, M3, Mstar, e12, e23, e13, b1_12, b1_23, b1_13, b2_12, b2_23, b2_13, alpha12, alpha23, alpha13, e1, e2, e3 = symbols("m1, m2, m3, Mstar, e_{12}, e_{23}, e_{13}, b1_{12}, b1_{23}, b1_{13}, b2_{12}, b2_{23}, b2_{13}, a_{12}, a_{23}, a_{13}, e_1, e_2, e_3", positive=True, real=True)
eps12, eps23, eps13 = symbols(r"\epsilon_{12}, \epsilon_{23}, \epsilon_{13}", positive=True, real=True)

G = Matrix([sqrt(M1)*e1, sqrt(M2)*e2, sqrt(M3)*e3])

In [8]:
# normalized coefficients in front of e1, e2, and e3 (i.e., unit vector in e1, e2 and e3 bases)
def convert(x):
    coeff1 = x.split("*e_1")[0]
    coeff1 = float(coeff1.replace(" ", ""))
    coeff2 = (x.split("*e_1")[1]).split("*e_2")[0]
    coeff2 = float(coeff2.replace(" ", ""))
    coeff3 = ((x.split("*e_1")[1]).split("*e_2")[1]).split("*e_3")[0]
    coeff3 = float(coeff3.replace(" ", ""))
    
    return 1/np.sqrt(coeff1**2+coeff2**2+coeff3**2) * np.array([coeff1, coeff2, coeff3])

In [9]:
def modeDistance(sim):
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M)
    exp_theo1 = (T.T*G.subs(M1, mass[0]).subs(M2, mass[1]).subs(M3, mass[2]))[0]
    vec_theo1 = convert(str(exp_theo1))
    exp_theo2 = (T.T*G.subs(M1, mass[0]).subs(M2, mass[1]).subs(M3, mass[2]))[1]
    vec_theo2 = convert(str(exp_theo2))
    
    vec_approx1, vec_approx2 = calcMode(sim)
    
    if vec_theo1[0] * vec_approx1[0] > 0:
        dist1 = np.linalg.norm(vec_theo1-vec_approx1)
    else:
        dist1 = np.linalg.norm(vec_theo1+vec_approx1)
    if vec_theo2[0] * vec_approx2[0] > 0:
        dist2 = np.linalg.norm(vec_theo2-vec_approx2)
    else:
        dist2 = np.linalg.norm(vec_theo2+vec_approx2)
    
    return dist1, dist2

In [10]:
def amp(x):
    return (max(x)-min(x))/(2*np.mean(x))

In [11]:
alpha = 0.85, 0.9
mass = 1e-7,1e-7,1e-7
ecc = 0.005, 0.005, 0.005
pomega = -np.pi/3, 0, np.pi/3
sim = make_sim(alpha, mass, ecc, pomega)

In [ ]:
mass_param = [[x,y] for x in np.array([1,3,10]) for y in np.array([1,3,10])]
alpha_param = [[x,y] for x in np.linspace(0.75,0.9,4) for y in np.linspace(0.75,0.9,4)]
ecc = 0.005, 0.005, 0.005
pomega = -np.pi/3, 0, np.pi/3

data = np.zeros((9*16,14))
for i, (alpha1, alpha2) in enumerate(alpha_param):
    for j, (m1, m3) in enumerate(mass_param):
        mass = m1*1e-7, 1e-7, m3*1e-7
        sim = make_sim((alpha1, alpha2), mass, ecc, pomega)
        dist1, dist2 = modeDistance(sim)
        psi_theo, psi_approx = calcPsi(sim)
        
        times, G1, G2, G3, S1, S2, S3, S1a, S2a, S3a = run(sim)
        amp_G1, amp_G2, amp_S1, amp_S2, amp_S1a, amp_S2a = amp(G1), amp(G2), amp(S1), amp(S2), amp(S1a), amp(S2a)
        
        data[i*9+j][:] = m1, m3, alpha1, alpha2, dist1, dist2, amp_G1, amp_G2, amp_S1, amp_S2, amp_S1a, amp_S2a, psi_theo, psi_approx

In [ ]:
pd.set_option('display.max_rows', 10)
# df = pd.DataFrame(data, columns=['m1','m3','alpha12','alpha23','psi_theo','psi_approx'], index=None)
df = pd.DataFrame(data, columns=['m1','m3','alpha12','alpha23','dist1','dist2','G1','G2','S1','S2','S1a','S2a','psi_theo','psi_approx'], index=None)
df

In [ ]:
df.to_csv("general.csv",index=False)

In [ ]:
mass_param = [[x,y] for x in np.array([1,3,10]) for y in np.array([1,3,10])]
alpha_param = [[0.9,0.9],[0.9,0.95],[0.95,0.9]]
ecc = 0.005, 0.005, 0.005
pomega = -np.pi/3, 0, np.pi/3

data = np.zeros((9*3,14))
for i, (alpha1, alpha2) in enumerate(alpha_param):
    for j, (m1, m3) in enumerate(mass_param):
        mass = m1*1e-9, 1e-9, m3*1e-9
        sim = make_sim((alpha1, alpha2), mass, ecc, pomega)
        dist1, dist2 = modeDistance(sim)
        psi_theo, psi_approx = calcPsi(sim)
        
        times, G1, G2, G3, S1, S2, S3, S1a, S2a, S3a = run(sim)
        amp_G1, amp_G2, amp_S1, amp_S2, amp_S1a, amp_S2a = amp(G1), amp(G2), amp(S1), amp(S2), amp(S1a), amp(S2a)
        
        data[i*9+j][:] = m1, m3, alpha1, alpha2, dist1, dist2, amp_G1, amp_G2, amp_S1, amp_S2, amp_S1a, amp_S2a, psi_theo, psi_approx

In [ ]:
pd.set_option('display.max_rows', 10)
# df = pd.DataFrame(data, columns=['m1','m3','alpha12','alpha23','psi_theo','psi_approx'], index=None)
df = pd.DataFrame(data, columns=['m1','m3','alpha12','alpha23','dist1','dist2','G1','G2','S1','S2','S1a','S2a','psi_theo','psi_approx'], index=None)
df

In [ ]:
df.to_csv("compact.csv",index=False)

In [ ]:
times, G1, G2, G3, S1, S2, S3, S1a, S2a, S3a = run(sim)

In [ ]:
fig, ax = plt.subplots()
ax.plot(times, G1/G1[0], color='blue', label='G1')
ax.plot(times, G2/G2[0], color='green', label='G2')
ax.plot(times, S1/S1[0], color='gray', linestyle='--', label='S1')
ax.plot(times, S1a/S1a[0], color='orange', linestyle='-.', label='S1a')
ax.legend()

# Test psi approximations

$$\sin{\psi} = \pm\sqrt{\frac{1}{2}-\frac{1}{2}\cdot\frac{(\tilde{m}_3-\tilde{m}_1\tilde{m}_2)b+(\tilde{m}_1-\tilde{m}_2\tilde{m}_3)a}{\sqrt{[(\tilde{m}_3+\tilde{m}_1\tilde{m}_2)b+(\tilde{m}_1+\tilde{m}_2\tilde{m}_3)a]^2-4ab\tilde{m}_2(\tilde{m}_1+\tilde{m}_3)^2}}}$$

$$\cos{\psi} = \pm\sqrt{\frac{1}{2}+\frac{1}{2}\cdot\frac{(\tilde{m}_3-\tilde{m}_1\tilde{m}_2)b+(\tilde{m}_1-\tilde{m}_2\tilde{m}_3)a}{\sqrt{[(\tilde{m}_3+\tilde{m}_1\tilde{m}_2)b+(\tilde{m}_1+\tilde{m}_2\tilde{m}_3)a]^2-4ab\tilde{m}_2(\tilde{m}_1+\tilde{m}_3)^2}}}$$

In [ ]:
def testPsi(sim):
    ps = sim.particles
    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    vals,T = np.linalg.eigh(M) # T returned by eigh is R^T in paper. Could define R = T.T
    Mdiag = T.T @ lsys.Neccentricity_matrix @ T
    
    ec13 = 1-ps[1].a/ps[3].a
    m1, m2, m3 = ps[1].m, ps[2].m, ps[3].m
    m_tot = m1 + m2 + m3
     
    mu1, mu2, mu3 = m1/m_tot, m2/m_tot, m3/m_tot
    alpha12, alpha23 = ps[1].a/ps[2].a, ps[2].a/ps[3].a
    alpha13 = alpha12*alpha23
    
    ec12 = alpha12**(-1/4)*alpha23**(3/4)*alpha23**(-1/8)*(1-alpha12)
    ec23 = alpha23**(-1/2)*alpha12**(1/8)*(1-alpha23)
    ec13 = alpha13**(-1/2)*(1-alpha13)
    
    R2 = np.array([[np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)],
                   [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
                   [np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)]])
    
    Mdiagapprox = R2 @ lsys.Neccentricity_matrix @ R2.T
    phi_theo = 1/2*np.arctan(2*Mdiagapprox[1,0]/(Mdiagapprox[1,1]-Mdiagapprox[0,0]))
    sin_theo, cos_theo = np.sin(phi_theo), np.cos(phi_theo)
    
    Delta = (ec12 - ec23)/ec13
    mu = (mu3-mu1)/(mu1+mu3)
    a = (Delta-1)**3*(3+Delta)
    b = (Delta+1)**3*(Delta-3)
    
    frac = 2*np.sqrt(mu2)*(a-b)/((a-mu2*b)*np.sqrt((1-mu)/(1+mu)) + (b-mu2*a)*np.sqrt((1+mu)/(1-mu)))
    phi_lim = 1/2*math.atan(frac)  # 1/2*atan(2k/w), this is the best we could approximate
    sin_lim, cos_lim = np.sin(phi_lim), np.cos(phi_lim)
    
    num = (mu3-mu1*mu2)*b + (mu1-mu2*mu3)*a
#     denom = ((mu3+mu1*mu2)*b + (mu1+mu2*mu3)*a)**2 - 4*a*b*mu2*(mu1+mu3)**2
    denom = (1-mu2)*((mu2+mu3)*b-(mu1+mu2)*a)
    K = num/denom

#     num = (b*(1 + m*(-1 + mu) + mu) - a*(-1 + m + mu + m*mu))
#     denom = -4*(a-b)**2*m*(-1 + mu**2)
#     print(denom/num**2)
#     K = 1-1/2*denom/num**2
    sin_approx, cos_approx = np.sqrt(1/2)*np.sqrt(1-K), np.sqrt(1/2)*np.sqrt(1+K)
    
    
    phi_saa = -np.sqrt(mu1*mu2*mu3/(mu1+mu3)**4)*Delta/(3/16+1/2*(mu3-mu1)*Delta+3/8*Delta**2-Delta**4/16)
    sin_saa, cos_saa = np.sin(phi_saa), np.cos(phi_saa)

    print('Theoretical:')
    print(sin_theo, cos_theo)
    print('Upper limit:')
    print(sin_lim, cos_lim)
    print('Approximation:')
    print(sin_approx, cos_approx)
    print('SAA:')
    print(sin_saa, cos_saa)